<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Data Preparation
Resave data as a set of tiff files in order to match Cell Tracking Challenge conventions which are expected by Baxter

In [1]:
import os
import sys

import numpy as np
from tifffile import imwrite

from deepcell.datasets import DynamicNuclearNetTracking
from deepcell_tracking.isbi_utils import trk_to_isbi

sys.path.append('..')
import utils

In [2]:
data_dir = 'data'
raw_dir = os.path.join(data_dir, 'raw')
seg_gt_dir = os.path.join(raw_dir, 'Analysis/Segmentation_GT')
gt_dir = os.path.join(data_dir, 'GT')

for d in [raw_dir, seg_gt_dir, gt_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

Load the test split of the tracking data

In [5]:
# Load test data
dnn = DynamicNuclearNetTracking(version='1.1')
X, y, lineages = dnn.load_data(split='test')
data = {
    'X': X,
    'y': y,
    'lineages': lineages
}

INFO:root:Checking for cached data
INFO:root:Making request to server
INFO:root:Downloading data/dynamic-nuclear-net/DynamicNuclearNet-tracking-v1_1.zip with size 2.6 GB to /Users/morganschwartz/.deepcell/datasets
2.62GB [05:23, 8.69MB/s]                             
INFO:root:🎉 Successfully downloaded file to /Users/morganschwartz/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip
INFO:root:Extracting /Users/morganschwartz/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip
INFO:root:Successfully extracted /Users/morganschwartz/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip into /Users/morganschwartz/.deepcell/datasets


Convert each batch of the test split to the standard ISBI format which is compatible with most of the models that we will test.

In [7]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    gt_subdir = os.path.join(gt_dir, '{:03}'.format(batch_no + 1))
    seg_gt_subdir = os.path.join(seg_gt_dir, '{:03}'.format(batch_no + 1))

    # Create directories if needed
    for d in (raw_subdir, gt_subdir, seg_gt_subdir):
        if not os.path.exists(d):
            os.makedirs(d)

    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]

    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]

    # Need to translate lineages and adjust images to match restrictive ISBI format
    # Prepare output txt
    text_file = os.path.join(gt_subdir, 'man_track.txt')
    df = trk_to_isbi(lineage)
    df.to_csv(text_file, sep=' ', header=False, index=False)

    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(X.shape[0]):
        name_raw = os.path.join(raw_subdir, 't{:03}.tif'.format(i))
        name_seg_gt = os.path.join(seg_gt_subdir, 't{:03}.tif'.format(i))
        name_gt = os.path.join(gt_subdir, 't{:03}.tif'.format(i))

        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_seg_gt, y[i, ..., channel].astype('uint16'))
        imwrite(name_gt, y[i, ..., channel].astype('uint16'))

2022-11-18 00:58:07.723153: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0. Please use `footprint` instead.
  markers = h_maxima(image=maxima,
/usr/local/lib/python3.8/dist-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima

# 2. Tracking

The steps for tracking will be performed in Matlab and the Baxter Algorithm GUI. You will need access to Matlab 2019b. The instructions below outline the general workflow for generating benchmarking results using the Baxter algorithm. However additional support for using the software is available in the user guide located in the KTH-SE folder (`KTH-SE/UserGuide/UserGuide.pdf`).

## Part 1 - Setup
1. Open the KTH-SE folder in Matlab
2. Open `BaxterAlgorithms.m` in the Matlab editor and then hit Run. This should open a GUI.
3. Within the GUI, select `Files > Open experiment > Browse...`. Navigate to the `data/raw` folder located within this Baxter folder.
4. Ensure that all files are selected in the list on the right hand side of the GUI.
## Part 2 - GT Segmentations
5. Select `Settings > Load Settings (Browse for files)`. Navigate to `Settings-SegmentationGT.csv` within the Baxter folder. In the following prompts, ensure that segmentations are applied to all images.
6. Select `Automated > Track`. In the pop-up window, change the `Save Version` to "GT" and press `Start`. 
7. Select `File > Export tracks to CTC format > RES tracks` and follow the prompts in order to export the tracking results to the standard CTC format. Make sure to select the "GT" version for export.


# 3. Evaluation

In [47]:
import os
import re
import shutil
import subprocess

import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [49]:
data_dir = 'data'
gt_dir = os.path.join(data_dir, 'GT')
export_gt_dir = os.path.join(data_dir, 'raw/Analysis/CellDataGT/RES/')
ctc_gt = os.path.join(data_dir, 'SEG_GT')
ctc_dc = os.path.join(data_dir, 'SEG_PRED')

data_ids = os.listdir(gt_dir)

node_match_threshold = 0.6

ctc_software = '../CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In order to utilize the CTC evaluation functions, the data must be reorganized to match CTC conventions.

In [32]:
# Copy the GT data over to appropriate folders
target = ctc_gt
for d in os.listdir(gt_dir):
    seg_dir = os.path.join(target, f'{d}_GT/SEG')
    tra_dir = os.path.join(target, f'{d}_GT/TRA')
    if not os.path.exists(seg_dir): os.makedirs(seg_dir)
    if not os.path.exists(tra_dir): os.makedirs(tra_dir)

    for f in os.listdir(os.path.join(gt_dir, d)):
        start_path = os.path.join(gt_dir, d, f)

        # Only copy the txt file with tracks to TRA dir
        if f.endswith('txt'):
            end_path = os.path.join(tra_dir, f)
            shutil.copy(start_path, end_path)
        else:
            # Copy to seg_dir and modify name
            end_path = os.path.join(seg_dir, f'man_seg{f[1:]}')
            shutil.copy(start_path, end_path)

            # Copy to tra dir and modify name
            end_path = os.path.join(tra_dir, f'man_track{f[1:]}')
            shutil.copy(start_path, end_path)

# Copy over results
target = ctc_gt
start_dirs = export_gt_dir
for d in os.listdir(start_dirs):
    start_dir = os.path.join(start_dirs, d)
    end_dir = os.path.join(target, d)
    shutil.copytree(start_dir, end_dir)

In [ ]:
benchmarks = []
results_dir = ctc_gt
s = 'GT'

for data_id in data_ids:
    print(data_id)
    results = {
        'model': f'Baxter - {s}',
        'data_id': data_id
    }
    gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
    res_dir = os.path.join(results_dir, f'{data_id}_RES')

    # Deepcell benchmarking
    m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
    results.update(m.stats)

    # CTC metrics
    for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
        p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                            stdout=subprocess.PIPE)
        outstring = p.stdout

        try:
            val = float(outstring.decode('utf-8').split()[-1])
            results[metric] = val
        except:
            print('Benchmarking failure', path, results_dir, data_id)
            print(outstring.decode('utf-8'))

    benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')